In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go 

def exponential(t, b):
    return np.exp(-b*np.asarray(t))

def polynomial(t, m, n ,o ,p):
    return m*t + n*t**2 + o*t**3 + p*t**4

def exponential_linear_comb(t, c, T1, T2):
    return c*np.exp(-np.asarray(t)/T1) + (1-c)*np.exp(-np.asarray(t)/T2)

df = pd.read_csv("./data/repair_times_OHL.csv")
data = df['Repair Times'].values.tolist()
#the data as extracted from the .csv will likely need to be reversed 
data.reverse()

Ft = np.arange(0,1.00001,1/(len(data)-1)) #this sets up a range of values for the repair time axis
x_axis = np.arange(0,data[0]+0.00001,0.1) #sets up an x_axis

params_comb, cov_comb = curve_fit(exponential_linear_comb, data, Ft)
params, covariances = curve_fit(exponential, data, Ft) #you only really care about the params, which
# are the variables you set in the mathematical function you defined
plt.scatter(data, Ft)
plt.plot(x_axis, exponential_linear_comb(x_axis, *params_comb), '--') #this will compare the data and the
# curve produced by the curve_fit function so you can judge how well it fits

In [ ]:
# x_axis = np.arange(0, len(data))  # Use the length of data as the range
params_poly, covariances_poly, = curve_fit(polynomial, data, Ft) 
params_comb, cov_comb = curve_fit(exponential_linear_comb, data, Ft)


# Plot using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=data, y=Ft, mode='markers', name='data'))
fig.add_trace(go.Scatter(x=x_axis, y=polynomial(x_axis, *params_poly), mode='lines', name='poly'))
fig.add_trace(go.Scatter(x=x_axis, y=exponential(x_axis, *params), mode='lines', name='exp'))
fig.add_trace(go.Scatter(x=x_axis, y=exponential_linear_comb(x_axis, *params_comb), mode='lines', name='comb exp'))
fig.update_layout(title='Polynomial Fit', xaxis_title='Repair Time', yaxis_title='F(t)')

fig.update_layout(width=800, height=800)
fig.show()
fig.write_image("./plots/exp_curves.png")

print(params_comb)
# Print mean square error of exponential_linear_comb
print("Mean Square Error of Exponential Linear Combination: ", np.mean((exponential_linear_comb(data, *params_comb) - Ft)**2))
# Print mean square error of exponential
print("Mean Square Error of Exponential: ", np.mean((exponential(data, *params) - Ft)**2))
